In [ ]:
#Using my puuid (Riot's account identifiation number) to get details of my account.
api_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/cDtzRYLS2a4M00LxENnNiYDPKuh3cREfZxIgPoY2Eiuuwy6aDQToCjAq9ikjVSvvV6I_eK21WiIzyA?api_key=RGAPI-9f49b300-e37d-44ca-ac9b-066e3642f525"
import requests 
requests.get(api_url)
resp2 = requests.get(api_url)
player_info2 = resp2.json()
print(player_info2)

In [4]:
#Using my puuid to request my most recent matches by ID, can only do so 100 at a time.
match_id_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/cDtzRYLS2a4M00LxENnNiYDPKuh3cREfZxIgPoY2Eiuuwy6aDQToCjAq9ikjVSvvV6I_eK21WiIzyA/ids?start=0&count=100&api_key=RGAPI-9f49b300-e37d-44ca-ac9b-066e3642f525"
import requests
requests.get(match_id_url)
resp3 = requests.get(match_id_url)
match_ids = resp3.json()
print(match_ids)
print(len(match_ids))

['NA1_5144442181', 'NA1_5144424872', 'NA1_5142715098', 'NA1_5142675303', 'NA1_5142654019', 'NA1_5140126301', 'NA1_5140110172', 'NA1_5139582018', 'NA1_5139571288', 'NA1_5139545203', 'NA1_5139506547', 'NA1_5139216815', 'NA1_5137608591', 'NA1_5137565062', 'NA1_5136598294', 'NA1_5136573342', 'NA1_5136549228', 'NA1_5135960525', 'NA1_5135246570', 'NA1_5135235325', 'NA1_5135225000', 'NA1_5133502016', 'NA1_5133483160', 'NA1_5133458533', 'NA1_5133435287', 'NA1_5131378450', 'NA1_5131352084', 'NA1_5131324169', 'NA1_5131305817', 'NA1_5131251500', 'NA1_5131225510', 'NA1_5130545606', 'NA1_5130523578', 'NA1_5128382028', 'NA1_5128351448', 'NA1_5126630949', 'NA1_5126599733', 'NA1_5125015081', 'NA1_5124975643', 'NA1_5124971955', 'NA1_5124944171', 'NA1_5124910007', 'NA1_5124702267', 'NA1_5124691813', 'NA1_5124670550', 'NA1_5124458182', 'NA1_5124443099', 'NA1_5124427148', 'NA1_5124401015', 'NA1_5121467475', 'NA1_5121433941', 'NA1_5121392935', 'NA1_5121353161', 'NA1_5121327952', 'NA1_5121311598', 'NA1_5121

In [8]:
#Using os to ensure my directory is set correctly and then connecting to MySQL database in order to insert the data. 
import os
os.chdir(r"C:\Users\AtomB\AppData\Local\Programs\Python\Python312\Lib\site-packages")
print(os.getcwd())
print(os.path.dirname(sys.executable))
import MySQLdb
import requests
db = MySQLdb.connect(host="localhost", user="root", password="", db="lol_database")
cursor = db.cursor()
starting_point = 0
end_point = 19
#This loop iterates through multiple matches, the API only allows for 20 requests every one seconds so no more than that can be done at once.
while starting_point <= end_point:
    #Constructing the URL to query the API, as well as instantiating a dictionary to hold all the data for a match. 
    match_id = match_ids[starting_point]
    match_url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key=RGAPI-9f49b300-e37d-44ca-ac9b-066e3642f525'
    requests.get(match_url)
    resp4 = requests.get(match_url)
    match_json = resp4.json()
    table_input = {
    "match_Id": "",
    "game_duration": 0,
    "start_timestamp": 0,
    "champ_name": "",
    "dam_to_buildings": 0,
    "kills": 0,
    "assists": 0,
    "deaths": 0,
    "kda": 0.0,
    "win": 0,
    "gold_earned": 0,
    "total_heal": 0,
    "total_dam_taken": 0,
    "total_dam_dealt": 0,
    "total_minions_killed": 0,
    "vision_score": 0
    }
    mypuuid = "cDtzRYLS2a4M00LxENnNiYDPKuh3cREfZxIgPoY2Eiuuwy6aDQToCjAq9ikjVSvvV6I_eK21WiIzyA"
    participant_selected = ""
    table_input["match_Id"] = match_json['metadata']['matchId']
    table_input["game_duration"] = match_json['info']['gameDuration']
    table_input["start_timestamp"] = match_json['info']['gameStartTimestamp']
    #Since all of the players in a match are listed and the position of my account will be random among the ten players in a match, this while loop checks the players for one 
    #that matches my unique puuid and then inserts all the desired data into my dictionary. 
    i = 0
    while i <= 6:
        participant_selected = match_json['info']['participants'][i]['puuid']
        if mypuuid == participant_selected:
            table_input["champ_name"] = match_json['info']['participants'][i]['championName']
            table_input["dam_to_buildings"] = match_json['info']['participants'][i]['damageDealtToBuildings']
            table_input["kills"] = match_json['info']['participants'][i]['kills']
            table_input["assists"] = match_json['info']['participants'][i]['assists']
            table_input["deaths"] = match_json['info']['participants'][i]['deaths']
            table_input["kda"] = match_json['info']['participants'][i]['challenges']['kda']
            table_input["win"] = match_json['info']['participants'][i]['win']
            table_input["gold_earned"] = match_json['info']['participants'][i]['goldEarned']
            table_input["total_heal"] = match_json['info']['participants'][i]['totalHeal']
            table_input["total_dam_taken"] = match_json['info']['participants'][i]['totalDamageTaken']
            table_input["total_dam_dealt"] = match_json['info']['participants'][i]['totalDamageDealt']
            table_input["total_minions_killed"] = match_json['info']['participants'][i]['totalMinionsKilled']
            table_input["vision_score"] = match_json['info']['participants'][i]['visionScore']
            break
        else:
            i += 1
    #Using the values from a dictionary I construct a SQL query to insert them into the table. 
    sql = "INSERT INTO lol_table VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    args = table_input["match_Id"], table_input["game_duration"], table_input["start_timestamp"], table_input["champ_name"], table_input["dam_to_buildings"], table_input["kills"], table_input["assists"], table_input["deaths"], table_input["kda"], table_input["win"], table_input["gold_earned"], table_input["total_heal"], table_input["total_dam_taken"], table_input["total_dam_dealt"], table_input["total_minions_killed"], table_input["vision_score"]
    cursor.execute(sql, args)
    starting_point += 1
#Commit and close the cursor and database connection after all the desired matches have been added. 
db.commit()
cursor.close()
db.close()

C:\Users\AtomB\AppData\Local\Programs\Python\Python312\Lib\site-packages
C:\Users\AtomB\anaconda3


In [ ]:
#This is largely the same as the one above, but with the difference that it was made to update old matches with the new data I wanted to collect, hence the use of UPDATE
#instead of INSERT. 
import os
import time
os.chdir(r"C:\Users\AtomB\AppData\Local\Programs\Python\Python312\Lib\site-packages")
print(os.getcwd())
print(os.path.dirname(sys.executable))
import MySQLdb
import requests
db = MySQLdb.connect(host="localhost", user="root", password="2k00l4u2bZ4cK!", db="lol_database")
cursor = db.cursor()
j = 40
while j <= 44:
    match_id = match_ids[j]
    match_url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key=RGAPI-9f49b300-e37d-44ca-ac9b-066e3642f525'
    requests.get(match_url)
    resp4 = requests.get(match_url)
    match_json = resp4.json()
    match_Id = ""
    gold_earned = 0
    total_heal = 0
    total_dam_t = 0
    total_dam_d = 0
    total_min_k= 0
    vis_sco = 0
    mypuuid = "cDtzRYLS2a4M00LxENnNiYDPKuh3cREfZxIgPoY2Eiuuwy6aDQToCjAq9ikjVSvvV6I_eK21WiIzyA"
    participant_selected = ""
    match_Id = match_json['metadata']['matchId']
    i = 0
    while i <= 9:
        participant_selected = match_json['info']['participants'][i]['puuid']
        if mypuuid == participant_selected:
            gold_earned = match_json['info']['participants'][i]['goldEarned']
            total_heal = match_json['info']['participants'][i]['totalHeal']
            total_dam_t = match_json['info']['participants'][i]['totalDamageTaken']
            total_dam_d = match_json['info']['participants'][i]['totalDamageDealt']
            total_min_k = match_json['info']['participants'][i]['totalMinionsKilled']
            vis_sco = match_json['info']['participants'][i]['visionScore']
            break
        else:
            i += 1
    sql = "UPDATE lol_table SET gold_earned = %s, total_heal = %s, total_dam_taken = %s, total_dam_dealt = %s, total_minions_killed = %s, vision_score = %s WHERE match_id = %s"
    args = gold_earned, total_heal, total_dam_t, total_dam_d, total_min_k, vis_sco, match_Id
    cursor.execute(sql, args)
    j += 1
db.commit()
cursor.close()
db.close()